# Shop Customer - Regression

[**Data by SHRUTI_IYYER from Kaggle**](https://www.kaggle.com/datasets/shrutimechlearn/customer-data)

A project to predict the spending score with the Age, Sex and Annual Income. I will try different regression model and compare their performance.

*Import the most important libraries*

In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

importing the Dataset

In [70]:
dataset = pd.read_csv('Customers.csv')
dataset.drop(columns=['CustomerID'], inplace=True)
dataset

,Genre,Age,Annual_Income_(k$),Spending_Score
0,Male,19,15,39
1,Male,21,15,81
2,Female,20,16,6
3,Female,23,16,77
4,Female,31,17,40
...,...,...,...,...
195,Female,35,120,79
196,Female,45,126,28
197,Male,32,126,74
198,Male,32,137,18


## Linear Regression

Data-preprocessing: selecting the Genre, Age and Annual Income as independent variables X and the Spending Score as dependent variable y.

In [71]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

Encode the categorical data of the Genre column, which has 'Female' and 'Male' as values. To avoid the dummy variable trap the first column of the transformed X is omitted. 

In [79]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
X = X[:, 1:]
X

array([[1.0, 19, 15],
       [1.0, 21, 15],
       [0.0, 20, 16],
       [0.0, 23, 16],
       [0.0, 31, 17],
       [0.0, 22, 17],
       [0.0, 35, 18],
       [0.0, 23, 18],
       [1.0, 64, 19],
       [0.0, 30, 19],
       [1.0, 67, 19],
       [0.0, 35, 19],
       [0.0, 58, 20],
       [0.0, 24, 20],
       [1.0, 37, 20],
       [1.0, 22, 20],
       [0.0, 35, 21],
       [1.0, 20, 21],
       [1.0, 52, 23],
       [0.0, 35, 23],
       [1.0, 35, 24],
       [1.0, 25, 24],
       [0.0, 46, 25],
       [1.0, 31, 25],
       [0.0, 54, 28],
       [1.0, 29, 28],
       [0.0, 45, 28],
       [1.0, 35, 28],
       [0.0, 40, 29],
       [0.0, 23, 29],
       [1.0, 60, 30],
       [0.0, 21, 30],
       [1.0, 53, 33],
       [1.0, 18, 33],
       [0.0, 49, 33],
       [0.0, 21, 33],
       [0.0, 42, 34],
       [0.0, 30, 34],
       [0.0, 36, 37],
       [0.0, 20, 37],
       [0.0, 65, 38],
       [1.0, 24, 38],
       [1.0, 48, 39],
       [0.0, 31, 39],
       [0.0, 49, 39],
       [0.

Splitting the dataset into a training set and test set. The test set will contain 40 values and the training set 160.

In [73]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Training the **Multiple Linear Regression** model on the training set. 

In [74]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

Predicting the results of the test set.

In [75]:
y_pred = regressor.predict(X_test)


In [81]:
X[:, 0]

array([1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0,
       0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0,
       0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,
       0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0,
       1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0,
       0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0,
       1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0,
       1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0,
       0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0,
       1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0,
       1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.

In [83]:
import statsmodels.api as sm
X_sm = np.column_stack((dataset['Age'], dataset['Annual_Income_(k$)']))
y_sm = dataset['Spending_Score']

X_sm = sm.add_constant(X_sm)
model = sm.OLS(y_sm, X_sm)
result = model.fit()
print(result.summary())


                            OLS Regression Results                            
Dep. Variable:         Spending_Score   R-squared:                       0.107
Model:                            OLS   Adj. R-squared:                  0.098
Method:                 Least Squares   F-statistic:                     11.82
Date:                Wed, 08 Mar 2023   Prob (F-statistic):           1.42e-05
Time:                        18:53:34   Log-Likelihood:                -922.21
No. Observations:                 200   AIC:                             1850.
Df Residuals:                     197   BIC:                             1860.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         73.3479      6.553     11.193      0.0

## Polynominal Regression

## Support Vector Regression (SVR)

## Decision Tree Regression 

## Random Forest Regression 

## **Comparison**